In [45]:
import pandas as pd

file_name = "data/parsed_psm/sc_rep5.tsv"
df = pd.read_csv(file_name, sep='\t', index_col=0)
df = df.sort_index()
df = df.set_index(["File Name","scan"]) # Not sure if we need to keep filename in the index. It depends on whether we get unique scans

In [46]:
df_series = df.drop(columns=["Matched Ion Intensities"])
df_intensities = df[["Matched Ion Intensities"]].copy()
# df_mz = df[["Matched Ion m/z"]].copy() # Or whater it turns out to be

# Deal with getting the Ion Name, Ion Charge, Ion Number (with respect to the peptide)
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.split(";")
df_series = df_series.explode("Matched Ion Series")
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.split(", ")
df_series = df_series.explode("Matched Ion Series")
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.strip("[]")
df_series = df_series.rename(columns={"Matched Ion Series": "Ion Name"})
df_series[["Ion Type", "Ion Number", "Ion Charge"]] = df_series["Ion Name"].str.extract(r"([abcxyz])(\d{1,3})([+-]\d)") # Tried to handle every possible case here, but this may need to be amended
df_series = df_series.astype({"Ion Charge": int, "Ion Number": int})
df_series = df_series.set_index("Ion Name", append=True)

# Deal with getting the Ion Intensity
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(";")
df_intensities = df_intensities.explode("Matched Ion Intensities")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(", ")
df_intensities = df_intensities.explode("Matched Ion Intensities")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.strip("[]")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(":")
df_intensities = pd.DataFrame(df_intensities["Matched Ion Intensities"].to_list(), columns=["Ion Name", "Intensity"], index=df_intensities.index)
df_intensities = df_intensities.set_index("Ion Name", append=True)
df_intensities = df_intensities.astype({"Intensity": float})

# Deal with getting the Ion m/z
# Will probably be very similar to getting the Ion intensity
# df_mz = ... 

df = df_series.join(df_intensities, how="inner")
# df = df_series.join(df_mz, how="inner")

df = df[["Ion Type", "Ion Number", "Ion Charge", "Intensity", "Base Peak Intensity", "peptide"]]
df

Ion Type  Ion Number  Ion Charge  \
File Name              scan  Ion Name                                    
D19_15um30cm_SC5-calib 14112 b2+1            b           2           1   
                             b3+1            b           3           1   
                             b4+1            b           4           1   
                             b5+1            b           5           1   
                             b6+1            b           6           1   
...                                        ...         ...         ...   
                       23659 y2+1            y           2           1   
                             y5+1            y           5           1   
                             y6+1            y           6           1   
                             y8+1            y           8           1   
                             b3+1            b           3           1   

                                       Intensity  Base Peak Intensity  \
File Name              scan  Ion Name                                   
D19_15um30cm_SC5-calib 14112 b2+1         3099.0          3432.428955   
                             b3+1         1096.0          3432.428955   
                             b4+1          569.0          3432.428955   
                             b5+1          545.0          3432.428955   
                             b6+1          890.0          3432.428955   
...                                          ...                  ...   
                       23659 y2+1          284.0          1418.214111   
                             y5+1          494.0          1418.214111   
                             y6+1          467.0          1418.214111   
                             y8+1          566.0          1418.214111   
                             b3+1         1101.0          1418.214111   

                                                                                 peptide  
File Name              scan  Ion Name                                                     
D19_15um30cm_SC5-calib 14112 b2+1                          AQGPAASAEEPKPVEAPAANSDQTVTVKE  
                             b3+1                          AQGPAASAEEPKPVEAPAANSDQTVTVKE  
                             b4+1                          AQGPAASAEEPKPVEAPAANSDQTVTVKE  
                             b5+1                          AQGPAASAEEPKPVEAPAANSDQTVTVKE  
                             b6+1                          AQGPAASAEEPKPVEAPAANSDQTVTVKE  
...                                                                                  ...  
                       23659 y2+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...  
                             y5+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...  
                             y6+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...  
                             y8+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...  
                             b3+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...  

[48014 rows x 6 columns]